In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [2]:
# Creating Model

drop_out_val = 0.05
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # CONVOLUTION LAYER 1                     
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=(3,3), padding=1)      
        self.bn1 = nn.BatchNorm2d(12)
        self.do1 = nn.Dropout2d(drop_out_val)
        
        # CONVOLUTION LAYER 2
        self.conv2 = nn.Conv2d(12, 24, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(24)
        self.do2 = nn.Dropout2d(drop_out_val)
        
        # CONVOLUTION LAYER 3
        self.conv3  = nn.Conv2d(24, 36, 3, padding=1)
        self.bn3    = nn.BatchNorm2d(36)
        self.do3    = nn.Dropout2d(drop_out_val)

        # TRANSITION LAYER 1
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv_t1 = nn.Conv2d(36, 12, 1)
        self.bn_t1 = nn.BatchNorm2d(12)
        self.do_t1 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 4
        self.conv4 = nn.Conv2d(12, 12, 3, padding = 1)
        self.bn4 = nn.BatchNorm2d(12)
        self.do4 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 5
        self.conv5  = nn.Conv2d(12, 24, 3, padding = 1)
        self.bn5    = nn.BatchNorm2d(24)
        self.do5    = nn.Dropout2d(drop_out_val)


        # TRANSITION LAYER 2
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv_t2  = nn.Conv2d(24, 12, 1)
        self.bn_t2    = nn.BatchNorm2d(12)
        self.do_t2    = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 6
        self.conv6 = nn.Conv2d(12, 12, 3, padding = 1)
        self.bn6 = nn.BatchNorm2d(12)
        self.do6 = nn.Dropout2d(drop_out_val)

        # CONVOLUTION LAYER 7
        self.conv7 = nn.Conv2d(12, 24, 3, padding = 1)

        self.gap = nn.AvgPool2d(7)
        self.linear1 = nn.Linear(24, 10)

    def forward(self, x):
        #BN after Relu
        x = self.pool1(self.do3(self.bn3(F.relu(self.conv3  (self.do2(self.bn2(F.relu(self.conv2  (self.do1(self.bn1(F.relu(self.conv1(x)))))  )))) ))))  
        x = self.pool2(self.do5(self.bn5(F.relu(self.conv5  (self.do4(self.bn4(F.relu(self.conv4  (self.do_t1(self.bn_t1(F.relu(self.conv_t1(x)))))  ))))  )))) 
        x =                       F.relu( self.conv7        (self.do6(self.bn6(F.relu(self.conv6  (self.do_t2(self.bn_t2(F.relu(self.conv_t2(x)))))  ))))  )               
        
        x = self.gap(x)
        x = x.view(-1, 24)
        x = self.linear1(x)

        return F.log_softmax(x)       #log of softmax


model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 28, 28]             120
       BatchNorm2d-2           [-1, 12, 28, 28]              24
         Dropout2d-3           [-1, 12, 28, 28]               0
            Conv2d-4           [-1, 24, 28, 28]           2,616
       BatchNorm2d-5           [-1, 24, 28, 28]              48
         Dropout2d-6           [-1, 24, 28, 28]               0
            Conv2d-7           [-1, 36, 28, 28]           7,812
       BatchNorm2d-8           [-1, 36, 28, 28]              72
         Dropout2d-9           [-1, 36, 28, 28]               0
        MaxPool2d-10           [-1, 36, 14, 14]               0
           Conv2d-11           [-1, 12, 14, 14]             444
      BatchNorm2d-12           [-1, 12, 14, 14]              24
        Dropout2d-13           [-1, 12, 14, 14]               0
           Conv2d-14           [-1, 12,

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [3]:
# Downloading Data

torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
# Function for Training & Validation

def train(model, device, train_loader, optimizer, scheduler, epoch):
    model.train()
    train_loss = 0
    correct = 0
    num_loops = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        num_loops +=1
 
    train_loss /= num_loops
    print("Training Average loss: {:.6f}, Accuracy = ({:.6f}%)".format(train_loss, 100. * correct / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.6f}, Accuracy: {}/{} ({:.6f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import ExponentialLR

model = Net().to(device)
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9,)

scheduler =ReduceLROnPlateau(optimizer=optimizer, patience=2, verbose=True)

for epoch in range(1, 10):
    print("Epoch: ", epoch)
    train(model, device, train_loader, optimizer, scheduler, epoch)
    print("learning rate", optimizer.param_groups[0]['lr'])
    test(model, device, test_loader)

optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
for epoch in range(10, 15):
    print("Epoch: ", epoch)
    train(model, device, train_loader, optimizer, scheduler, epoch)
    print("learning rate", optimizer.param_groups[0]['lr'])
    test(model, device, test_loader)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
for epoch in range(15, 20):
    print("Epoch: ", epoch)
    train(model, device, train_loader, optimizer, scheduler, epoch)
    print("learning rate", optimizer.param_groups[0]['lr'])
    test(model, device, test_loader)

Epoch:  1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Training Average loss: 0.320848, Accuracy = (89.638333%)
learning rate 0.1
Test set: Average loss: 0.066719, Accuracy: 9796/10000 (97.960000%)

Epoch:  2
Training Average loss: 0.098134, Accuracy = (97.021667%)
learning rate 0.1
Test set: Average loss: 0.044603, Accuracy: 9863/10000 (98.630000%)

Epoch:  3
Training Average loss: 0.078118, Accuracy = (97.598333%)
learning rate 0.1
Test set: Average loss: 0.041200, Accuracy: 9863/10000 (98.630000%)

Epoch:  4
Training Average loss: 0.067053, Accuracy = (97.973333%)
learning rate 0.1
Test set: Average loss: 0.050660, Accuracy: 9834/10000 (98.340000%)

Epoch:  5
Training Average loss: 0.059125, Accuracy = (98.185000%)
learning rate 0.1
Test set: Average loss: 0.040517, Accuracy: 9876/10000 (98.760000%)

Epoch:  6
Training Average loss: 0.054698, Accuracy = (98.361667%)
learning rate 0.1
Test set: Average loss: 0.033523, Accuracy: 9881/10000 (98.810000%)

Epoch:  7
Training Average loss: 0.050320, Accuracy = (98.486667%)
learning rate 0.1
T